# Analyzing IMDB Data in Keras - Solution

In [24]:
# Imports
import numpy as np
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)

## 1. Loading the data
This dataset comes preloaded with Keras, so one simple command will get us training and testing data. There is a parameter for how many words we want to look at. We've set it at 1000, but feel free to experiment.

In [25]:
# Loading the data (it's preloaded in Keras)
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=1000, skip_top=10)

print(x_train.shape)
print(x_test.shape)
print(x_train[857])
print(y_train[857])
print(len(x_train[857]))

(25000,)
(25000,)
[2, 32, 11, 32, 2, 52, 22, 2, 128, 18, 2, 192, 15, 69, 2, 22, 24, 77, 93, 2, 65, 238, 2, 2, 2, 2, 2, 2, 127, 2, 52, 292, 17, 2, 2, 298, 19, 2, 2, 501, 2, 2, 2, 2, 2, 2, 49, 14, 203, 30, 99, 2, 2, 409, 357, 18, 72, 12, 2, 2, 2, 73, 579, 787, 2, 404, 18, 223, 829, 150, 198, 24, 142, 25, 79, 2, 176, 2, 2]
1
79


## 2. Examining the data
Notice that the data has been already pre-processed, where all the words have numbers, and the reviews come in as a vector with the words that the review contains. For example, if the word 'the' is the first one in our dictionary, and a review contains the word 'the', then there is a 1 in the corresponding vector.

The output comes as a vector of 1's and 0's, where 1 is a positive sentiment for the review, and 0 is negative.

In [26]:
print(x_train[0])
print(y_train[0])

[2, 14, 22, 16, 43, 530, 973, 2, 2, 65, 458, 2, 66, 2, 2, 173, 36, 256, 2, 25, 100, 43, 838, 112, 50, 670, 2, 2, 35, 480, 284, 2, 150, 2, 172, 112, 167, 2, 336, 385, 39, 2, 172, 2, 2, 17, 546, 38, 13, 447, 2, 192, 50, 16, 2, 147, 2, 19, 14, 22, 2, 2, 2, 469, 2, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 2, 2, 22, 17, 515, 17, 12, 16, 626, 18, 2, 2, 62, 386, 12, 2, 316, 2, 106, 2, 2, 2, 2, 16, 480, 66, 2, 33, 2, 130, 12, 16, 38, 619, 2, 25, 124, 51, 36, 135, 48, 25, 2, 33, 2, 22, 12, 215, 28, 77, 52, 2, 14, 407, 16, 82, 2, 2, 2, 107, 117, 2, 15, 256, 2, 2, 2, 2, 2, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 2, 2, 2, 2, 13, 104, 88, 2, 381, 15, 297, 98, 32, 2, 56, 26, 141, 2, 194, 2, 18, 2, 226, 22, 21, 134, 476, 26, 480, 2, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 2, 226, 65, 16, 38, 2, 88, 12, 16, 283, 2, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]
1


## 3. One-hot encoding the output
Here, we'll turn the input vectors into (0,1)-vectors. For example, if the pre-processed vector contains the number 14, then in the processed vector, the 14th entry will be 1.

In [27]:
# Turning the output into vector mode, each of length 1000
tokenizer = Tokenizer(num_words=1000)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print(x_train.shape)
print(x_test.shape)
#print(x_train[0])

(25000, 1000)
(25000, 1000)


And we'll one-hot encode the output.

In [28]:
# One-hot encoding the output
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)
print(y_test.shape)

(25000, 2)
(25000, 2)


## 8. Create a CNN

In [78]:
'''from keras.layers import Conv2D, MaxPooling2D, MaxPooling1D, Flatten
modelC = Sequential()
print(x_train.shape)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1, 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1, 1)
print(x_train.shape[1:])
modelC.add(Conv2D(32, (1, 1), activation='relu', input_shape=x_train.shape[1:]))
modelC.add(Conv2D(32, (1, 1), activation='relu'))
modelC.add(MaxPooling1D(pool_size=2))
modelC.add(Dropout(0.25))

modelC.add(Flatten())
modelC.add(Dense(256, activation='relu'))
modelC.add(Dropout(0.5))
modelC.add(Dense(num_classes, activation='softmax'))
modelC.summary()

modelC.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='saved_models/weights.cnnModel.hdf5',
                               save_best_only=True)

modelC.fit(x_train, y_train,
          batch_size=32,
          epochs=10,
          validation_data=(x_test, y_test), callbacks=[checkpointer],
          verbose=2)

modelC.load_weights('saved_models/weights.cnnModel.hdf5')
score = modelC.evaluate(x_test, y_test, verbose=0)
print("accuracy: ", score[1])
'''


(25000, 1000, 1, 1)
(1000, 1, 1)


ValueError: Input 0 is incompatible with layer max_pooling1d_7: expected ndim=3, found ndim=4